# Spam Filter for SMS messages using Naive Bayes

The aim of this project, to classify messages as spam or non-spam, we saw in the previous mission that the computer:

1. Learns how humans classify messages.
2. Uses that human knowledge to estimate probabilities for new messages probabilities for spam and non-spam.
3. Classifies a new message based on these probability values if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

In [1]:
import pandas as pd
sms_spam = pd.read_csv("SMSSpamCollection", sep = '\t',header = None, names = ['Label', 'SMS'])

print(sms_spam.shape)
sms_spam.head()


(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


"ham" means non-spam

In [2]:
sms_spam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

- 87% of sms are non-spam.
- 13% of sms are spam

# Randomize the dataset

For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham.

## Training and Test Dataset

- Split the dataset into a training and test dataset
- 80% training and 20% test dataset


In [3]:
data = sms_spam.sample(random_state=1, frac=1)

# identifing index to split data
# 80% data
spliting_index = round(len(data)*0.8) 

# training dataset: upto spliting index
# test dataset: from spliting index
training_set = data[:spliting_index].reset_index(drop=True)
test_set = data[spliting_index:].reset_index(drop=True)


In [4]:
print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [5]:
print("Training Dataset")
print(training_set['Label'].value_counts(normalize=True))
print("\n")
print("Test dataset")
print(test_set['Label'].value_counts(normalize=True))

Training Dataset
ham     0.86541
spam    0.13459
Name: Label, dtype: float64


Test dataset
ham     0.868043
spam    0.131957
Name: Label, dtype: float64


------

# Data Cleaning
- To bring the data in a format that will allow us to extract easily all the information we need.

Tasks to do:

1. Letter Case and Punctuation
    Remove all the punctuation and bring every letter to lower case.
2. Creating the Vocabulary
    A list with all the unique words in our training set.
3. Final Training Set
    Use the created vocabulary to make the data transformation we want
-----
### 1. Letter Case and Punctuation

In [6]:
# check the dataset
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [7]:
# use \W to detect any character that is not from a-z, A-Z or 0-9.
training_set['SMS']= training_set['SMS'].str.replace('\W',' ')

#lower case
training_set['SMS']= training_set['SMS'].str.lower()

training_set.head()



,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### 2. Creating the Vocabulary

In [8]:
training_set['SMS'] = training_set['SMS'].str.split()

vocab = []

for sms in training_set['SMS']:
    for word in sms:
        vocab.append(word)
    
vocabulary = list(set(vocab))
    

In [9]:
len(vocabulary)

7783

### 3. The Final training set

In [10]:
# create an empty dictionary named "word_counts_per_sms"
# where key --> unique word, value --> 0
# [0]* 5 i.e. [0,0,0,0,0]
# here {(uw1,0),(uw2,0),..(last uw,0)}

word_counts_per_sms = {unique_word:[0] * len(training_set['SMS']) for unique_word in vocabulary}

# loop over index and SMS to count the occurence of word
# enumerate() function to get both the index and the SMS message

for index,sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] +=1


In [11]:
# Transform word_counts_per_sms into a DataFrame 

word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [ ]:
# Concatenate the DataFrame we just built above with the DataFrame containing the training set. This way, 
# we'll also have the Label and the SMS columns.
# axis = 1 to concatenate as column

final_training_data = pd.concat([training_set,word_counts], axis =1)
final_training_data.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


- NSpam is equal to the number of words in all the spam messages — it's not equal to the number of spam messages, and it's not equal to the total number of unique words in spam messages.
- NHam is equal to the number of words in all the non-spam messages — it's not equal to the number of non-spam messages, and it's not equal to the total number of unique words in non-spam messages.
- NVocabulary is len(vocabulary)
- alpha = 1

In [ ]:
# P(Spam) and P(Ham)--> normalised value counts
#------------------------------------
p_spam = final_training_data['Label'].value_counts(normalize=True)['spam']
p_ham = final_training_data['Label'].value_counts(normalize=True)['ham']

# N_Spam, N_Ham, and N_Vocabulary -->
#-----------------------------
n_spam = final_training_data[final_training_data['Label']=="spam"].sum(axis = 1).sum()
n_ham = final_training_data[final_training_data['Label']=="ham"].sum(axis = 1).sum()
n_vocubulary = len(vocabulary)
alpha = 1


In [ ]:
# P(Spam) and P(Ham)--> normalised value counts
#------------------------------------
p_spam = final_training_data['Label'].value_counts(normalize=True)['spam']
p_ham = final_training_data['Label'].value_counts(normalize=True)['ham']

